# U_former - <font color="red">Denoise</font>
***

Uformer, an image restoration model, based on Transformer Architecture aims to leverage the capability of self-attention in feature maps at multi-scale resolutions to recover more image details. 

Denoising in image restoration refers to the process of removing unwanted or random variations, known as noise, from a digital image which aims to enhance the quality and clarity of an image by reducing or eliminating this noise while preserving the important underlying image features.

## Imports

In [1]:
import os
import sys

notebook_dir = os.getcwd()
dir_name = os.path.join(notebook_dir)
sys.path.append(os.path.join(dir_name, './development'))
directory = os.path.join(dir_name, './development/') 

print(notebook_dir)
print(dir_name)
print(sys.path)
print(directory) 

D:\YEDHU_PROJECT\nn_project_uformer
D:\YEDHU_PROJECT\nn_project_uformer
['D:\\YEDHU_PROJECT\\nn_project_uformer', 'C:\\Users\\SARATHCHANDRAKUMAR\\AppData\\Local\\Programs\\Python\\Python37\\python37.zip', 'C:\\Users\\SARATHCHANDRAKUMAR\\AppData\\Local\\Programs\\Python\\Python37\\DLLs', 'C:\\Users\\SARATHCHANDRAKUMAR\\AppData\\Local\\Programs\\Python\\Python37\\lib', 'C:\\Users\\SARATHCHANDRAKUMAR\\AppData\\Local\\Programs\\Python\\Python37', 'C:\\venvs\\uformer', '', 'C:\\venvs\\uformer\\lib\\site-packages', 'C:\\venvs\\uformer\\lib\\site-packages\\win32', 'C:\\venvs\\uformer\\lib\\site-packages\\win32\\lib', 'C:\\venvs\\uformer\\lib\\site-packages\\Pythonwin', 'C:\\venvs\\uformer\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\SARATHCHANDRAKUMAR\\.ipython', 'D:\\YEDHU_PROJECT\\nn_project_uformer\\./development']
D:\YEDHU_PROJECT\nn_project_uformer\./development/


In [2]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

C:\venvs\uformer\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA is available. Using GPU: NVIDIA GeForce GTX 1050


In [3]:
import torch.nn as nn
import utils
import torch.optim as optim
import datetime
import time
import matplotlib.pyplot as plt
import logging
import argparse
import options
import math

from model import Uformer
from torch.optim.lr_scheduler import StepLR
from losses import CharbonnierLoss
from torch.utils.data import DataLoader
from utils.loader import get_training_data, get_validation_data
from dataset import get_validation_deblur_data
from tqdm import tqdm
from timm.utils import NativeScaler
from warmup_scheduler import GradualWarmupScheduler
from skimage.metrics import peak_signal_noise_ratio as psnr_loss
from skimage.metrics import structural_similarity as ssim_loss
from skimage import img_as_float32, img_as_ubyte
from dataset.dataset_denoise import *
from model import UNet,Uformer

## Data Preparation
The data within SIDD Medium dataset is patches in to custom made train, validation, test datasets using `custom_dataset_denoise.py`. The generated patch data sets are stored in `/deployment/dataset/denoise/SIDD/customized_dataset` with separate folders and each of the datasets contains a ground_truth and an input folders.

## Setting Directories 

In [4]:
pretrain_weights_path=os.path.join(directory, "./models/pretrained/denoise/model_best.pth")
train_dir=os.path.join(directory, "./datasets/denoise/SIDD/customized_dataset/train")
val_dir=os.path.join(directory, "./datasets/denoise/SIDD/customized_dataset/val")
test_dir=os.path.join(directory, "./datasets/denoise/SIDD/customized_dataset/test")
model_dir=os.path.join(directory, "./models/training/denoise")

## Setting Hyperparameters

In [5]:
train_ps = 128   # train patch size
val_ps = 128   # validation patch size
test_ps = 128   # test patch size
dd_in = 3   #dd_in
optimizer = 'adamw'
lr_initial = 0.0002   # learning rate
weight_decay = 0.02
warmup_epochs = 2
pretrain_weights = pretrain_weights_path
train_workers = 4
eval_workers = 4
checkpoint = 50
batch_size = 1
nepoch = 3   # number of epochs for training
resume = True
do_validation= True
warmup = True
embed_dim=32
win_size=8
checkpoint = 50


#NOTE:
#nepoch != warmup_epochs ==> causes error in scheduler.step()


## Dataset loading
`Training` and `validation` datasets are loaded using `get_training_data` and `get_validation_data`. The data is organized into batches using the DataLoader class, which provides parallel data loading and preprocessing. For the training dataset, shuffling is enabled to enhance randomness during training. For validation, shuffling is turned off to ensure consistent evaluation.

In [6]:
print('===> Loading datasets')
img_options_train = {'patch_size':train_ps}
train_dataset = get_training_data(train_dir, img_options_train)
train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True,
                          num_workers=train_workers, 
                          pin_memory=False, 
                          drop_last=False)
val_dataset = get_validation_data(val_dir)
val_loader = DataLoader(dataset=val_dataset,
                        batch_size=batch_size, 
                        shuffle=False, 
                        num_workers=eval_workers, 
                        pin_memory=False, 
                        drop_last=False)
test_dataset = get_validation_data(test_dir)
test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, 
                         shuffle=False, 
                         drop_last=False)

len_trainset = train_dataset.__len__()
len_valset = val_dataset.__len__()
len_testset = test_dataset.__len__()
print("Size of Train Dataset:{}\nSize of Validation Dataset:{}\nSize of Test Dataset:{} "
      .format(len_trainset,len_valset,len_testset))

===> Loading datasets
Size of Train Dataset:40
Size of Validation Dataset:40
Size of Test Dataset:40 


## Loading Model Architecture
Loading the Uformer architecture with the hyper parameters into model_restoration

In [7]:
model_restoration = Uformer(img_size=train_ps,
                            embed_dim=embed_dim,
                            win_size=8,
                            token_projection='linear',
                            token_mlp='leff',
                            depths=[1, 2, 8, 8, 2, 8, 8, 2, 1],
                            modulator=True,
                            dd_in=dd_in) 

C:\venvs\uformer\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


## Setting Optimizer & Loss
Creating an AdamW optimizer for model parameter optimization and setting CharbonnierLoss as the loss function and move it to GPU

In [8]:
optimizer = optim.AdamW(model_restoration.parameters(),
                        lr=lr_initial,
                        betas=(0.9, 0.999),
                        eps=1e-8,
                        weight_decay=weight_decay)
print (optimizer)
criterion = CharbonnierLoss().cuda()

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    lr: 0.0002
    maximize: False
    weight_decay: 0.02
)


## Setting Data Parallel
Configure the model to use data parallelism for efficient utilization of multiple GPUs for faster training.

In [9]:
model_restoration = torch.nn.DataParallel (model_restoration) 
model_restoration.cuda();

## Setting Scheduler
If the `warmup` flag is enabled, a combination of warmup and cosine annealing to gradually adjust the learning rate. Alternatively, if the `warmup` flag is not enabled,a step-based strategy using the StepLR scheduler is using. 

In [10]:
if warmup:
    print("Using warmup and cosine strategy!")
    warmup_epochs = warmup_epochs
    scheduler_cosine = optim.lr_scheduler.CosineAnnealingLR(optimizer,nepoch-warmup_epochs, eta_min=1e-6)
    scheduler = GradualWarmupScheduler(optimizer, multiplier=1, total_epoch=warmup_epochs, after_scheduler=scheduler_cosine)
    scheduler.step()
else:
    step = 50
    print("Using StepLR,step={}!".format(step))
    scheduler = StepLR(optimizer, step_size=step, gamma=0.5)
    scheduler.step()

Using warmup and cosine strategy!


C:\venvs\uformer\lib\site-packages\torch\optim\lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


##  Setting Resume
If the `resume` option is enabled, the training is resume from the checkpoint. This involves loading the model's previous state, optimizer parameters, and the starting epoch and the learning rate scheduler is updated to reflect the training progress up to the resumed epoch.

In [12]:
if resume: 
    path_chk_rest = pretrain_weights 
    print("Resume from "+path_chk_rest)
    utils.load_checkpoint(model_restoration,path_chk_rest) 
    start_epoch = utils.load_start_epoch(path_chk_rest) + 1 
    lr = utils.load_optim(optimizer, path_chk_rest)
    print("start epoch: ",start_epoch)
    for i in range(1, start_epoch):
        scheduler.step()
    new_lr = scheduler.get_last_lr()[0]
    print("===> Resuming Training with learning rate:", new_lr)
    nepoch=start_epoch+nepoch-1
    print(f"end epoch: {nepoch}")
else:
    start_epoch = 1

Resume from D:\YEDHU_PROJECT\nn_project_uformer\./development/./models/pretrained/denoise/model_best.pth
start epoch:  50
===> Resuming Training with learning rate: 0.0002
end epoch: 52


C:\venvs\uformer\lib\site-packages\torch\optim\lr_scheduler.py:808: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


## Model Validation
The restored model is validate using the validation dataset (val_loader). For each batch of validation data, the model's evaluation mode is set using .eval(), and the input data is passed through the restoration model to obtain the restored output. The PSNR is computed between the input and ground_truth images, as well as between the restored output and the ground_truth. After processing all validation batches, the average PSNR values for the dataset and the model's initial output are computed and stored.

In [13]:
if do_validation :
    with torch.no_grad():
        model_restoration.eval()
        psnr_dataset = []
        psnr_model_init = []
        for ii, data_val in enumerate(tqdm(val_loader ), 0):
            target = data_val[0].cuda()
            input_ = data_val[1].cuda()
            with torch.cuda.amp.autocast():
                restored = model_restoration(input_)
                restored = torch.clamp(restored,0,1)  
            psnr_dataset.append(utils.batch_PSNR(input_, target, False).item())
            psnr_model_init.append(utils.batch_PSNR(restored, target, False).item())
        psnr_dataset = sum(psnr_dataset)/len_valset
        psnr_model_init = sum(psnr_model_init)/len_valset
        print('PSNR: Input & GT =>%.4f dB'%(psnr_dataset), '\nPSNR: Model_init & GT =>%.4f dB'%(psnr_model_init))

100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:19<00:00,  2.07it/s]

PSNR: Input & GT =>32.1453 dB 
PSNR: Model_init & GT =>47.3363 dB


## Model Training
Iterating through epochs and batches for training the `restoration_model`. Inside the training loop, calculate the loss using the given `criterion` and perform backpropagation to update the model's parameters. At regular intervals specified by `eval_now`, compute PSNR values on the validation dataset, and save the model's best weights if the PSNR improves. After each epoch, adjust the `learning rate` using the `scheduler`. Model checkpoints are saved at regular intervals and at the best PSNR epoch.

In [ ]:
print('===> Start Epoch {} End Epoch {}'.format(start_epoch, nepoch))
best_psnr = 0
best_epoch = 0
best_iter = 0
eval_now = len(train_loader)//4  
print("\nEvaluation after every {} Iterations !!!\n".format(eval_now))

loss_scaler = NativeScaler()
torch.cuda.empty_cache()
start_time = time.time()   
psnr_train_rgb_epoch=[]
psnr_val_best_rgb_epoch=[]
for epoch in range(start_epoch, nepoch+1):
    epoch_loss = 0
    train_id = 1
    psnr_train_rgb = []
    for i, data in enumerate(tqdm(train_loader), 0): 
        optimizer.zero_grad()
        target = data[0].cuda()
        input_ = data[1].cuda()
        with torch.cuda.amp.autocast():
            restored = model_restoration(input_)
            loss = criterion(restored, target)  
        restored = torch.clamp(restored,0,1) 
        psnr_train_rgb.append(utils.batch_PSNR(restored, target, False).item())     
        loss_scaler(loss, optimizer,parameters=model_restoration.parameters())
        epoch_loss +=loss.item()
        # Evaluation #
        if (i+1)%eval_now==0 and i>0:
            with torch.no_grad():
                model_restoration.eval()
                psnr_val_rgb = []
                for ii, data_val in enumerate((val_loader), 0):
                    target = data_val[0].cuda()
                    input_ = data_val[1].cuda()
                    filenames = data_val[2]
                    with torch.cuda.amp.autocast():
                        restored = model_restoration(input_)
                    restored = torch.clamp(restored,0,1)  
                    psnr_val_rgb.append(utils.batch_PSNR(restored, target, False).item())     
                psnr_val_rgb = sum(psnr_val_rgb)/len_valset

                # calculate best PSNR
                if psnr_val_rgb > best_psnr:
                    best_psnr = psnr_val_rgb
                    best_epoch = epoch
                    best_iter = i 
                    torch.save({'epoch': epoch, 
                                'state_dict': model_restoration.state_dict(),
                                'optimizer' : optimizer.state_dict()
                                }, os.path.join(model_dir,"model_best.pth"))                    
    psnr_train_rgb=sum(psnr_train_rgb)/len_trainset
    psnr_train_rgb_epoch.append(psnr_train_rgb)
    psnr_val_best_rgb_epoch.append(best_psnr)

    print(f"Epoch:{epoch}, Loss:{epoch_loss:.4f}, PSNR_train:{psnr_train_rgb:.4f}dB, PSNR_val:{best_psnr:.4f}dB")
    scheduler.step()
    torch.save({'epoch': epoch, 
                'state_dict': model_restoration.state_dict(),
                'optimizer' : optimizer.state_dict()
                }, os.path.join(model_dir,"model_latest.pth"))   
    if epoch%checkpoint == 0:
          torch.save({'epoch': epoch, 
                    'state_dict': model_restoration.state_dict(),
                    'optimizer' : optimizer.state_dict()
                    }, os.path.join(model_dir,"model_epoch_{}.pth".format(epoch)))
    torch.cuda.empty_cache()
    
end_time = time.time()

# time calculation
formatted_start_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(start_time))
formatted_end_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(end_time))

t_ime = end_time-start_time
total_seconds = int(t_ime)
seconds = total_seconds % 60
total_minutes = total_seconds // 60
minutes = total_minutes % 60
total_hours = total_minutes // 60
hours = total_hours % 24
days = total_hours // 24

print("------------------------------------------------------------------")
print("Training Completed...")
print(f"PSNR TRAIN RGB : {(sum(psnr_train_rgb_epoch)/len(psnr_train_rgb_epoch)):.4f}dB")
print(f"PSNR VAL RGB : {(sum(psnr_val_best_rgb_epoch)/len(psnr_val_best_rgb_epoch)):.4f}dB")
print("------------------------------------------------------------------")
print("Train Start:{}\nTrain End:{}\nTraining Time: {} days, {} Hs, {} Ms, {} S "
      .format(formatted_start_time,formatted_end_time,days,hours,minutes,seconds))


===> Start Epoch 50 End Epoch 52

Evaluation after every 10 Iterations !!!



100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [02:02<00:00,  3.06s/it]


Epoch:50, Loss:0.1328, PSNR_train:48.5432dB, PSNR_val:47.3481dB


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [02:09<00:00,  3.23s/it]


Epoch:51, Loss:0.1370, PSNR_train:48.1062dB, PSNR_val:47.3481dB


 28%|██████████████████████▌                                                           | 11/40 [00:38<02:35,  5.35s/it]

## Conversion to square images

The model deals with only square images and hence it should be taken care about the imput image given to the testing module. The `expand2square` function takes a PyTorch image tensor and resizes it to a square shape while maintaining its original content. It calculates a target size based on a given factor, then creates an expanded image tensor and mask. The original image is centered within the new canvas, and a mask marks the valid regions. The function returns the resized image tensor and mask.

In [17]:
def expand2square(timg,factor=16.0,ps=1):
    _, _, h, w = timg.size()
    X = int(math.ceil(max(h,w)/float(factor))*factor)
    X = math.ceil(X/ps)*ps
    img = torch.zeros(1,3,X,X).type_as(timg) # 3, h,w
    mask = torch.zeros(1,1,X,X).type_as(timg)
    img[:,:, ((X - h)//2):((X - h)//2 + h),((X - w)//2):((X - w)//2 + w)] = timg
    mask[:,:, ((X - h)//2):((X - h)//2 + h),((X - w)//2):((X - w)//2 + w)].fill_(1)
    return img, mask

## Setting Directories & Hyperparameters for testing


In [18]:
trained_path=os.path.join(directory,"./models/training/denoise/model_best.pth")
result_dir=os.path.join(directory,"./result_dir/testing/denoise")

trained_weights = trained_path
batch_size = 1

if os.path.exists(result_dir):
    print("Result directory already exist.")
else:
    utils.mkdir(result_dir)
    print("Result directory created at {}".format(result_dir))

Result directory already exist.
trained_path: D:\YEDHU_PROJECT\./CODE/deployment/./models/training/denoise/model_best.pth
result_dir: D:\YEDHU_PROJECT\./CODE/deployment/./result_dir/testing/denoise


## Loading model for testing
A Uformer architecture is created for the testing and is loaded with the currently generated weights during training.

In [27]:
model_testing = Uformer(img_size=test_ps,
                            embed_dim=embed_dim,
                            win_size=8,
                            token_projection='linear',
                            token_mlp='leff',
                            depths=[1, 2, 8, 8, 2, 8, 8, 2, 1],
                            modulator=True,
                            dd_in=dd_in).cuda() 
utils.load_checkpoint(model_testing,trained_weights)
print("===>Testing using weights: ", trained_weights)

===>Testing using weights:  D:\YEDHU_PROJECT\./CODE/deployment/./models/training/denoise/model_best.pth


## Model Validation
The restored model is validate using the validation dataset (val_loader). For each batch of validation data, the model's evaluation mode is set using .eval(), and the input data is passed through the restoration model to obtain the restored output. The PSNR is computed between the input and ground_truth images, as well as between the restored output and the ground_truth. After processing all validation batches, the average PSNR values for the dataset and the model's initial output are computed and stored.

In [28]:
if do_validation :
    with torch.no_grad():
        model_testing.eval()
        psnr_dataset = []
        psnr_model_init = []
        for ii, data_val in enumerate(tqdm(test_loader ), 0):
            target = data_val[0].cuda()
            input_ = data_val[1].cuda()
            with torch.cuda.amp.autocast():
                restored = model_testing(input_)
                restored = torch.clamp(restored,0,1)  
            psnr_dataset.append(utils.batch_PSNR(input_, target, False).item())
            psnr_model_init.append(utils.batch_PSNR(restored, target, False).item())
        psnr_dataset = sum(psnr_dataset)/len_valset
        psnr_model_init = sum(psnr_model_init)/len_valset
        print('PSNR: Input & GT =>%.4f dB'%(psnr_dataset), '\nPSNR: Model_init & GT =>%.4f dB'%(psnr_model_init))

100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:15<00:00,  2.55it/s]

PSNR: Input & GT =>32.5485 dB 
PSNR: Model_init & GT =>47.0243 dB


## Model Testing

In [31]:
with torch.no_grad():
    psnr_val_rgb = []
    ssim_val_rgb = []
    for ii, data_test in enumerate(tqdm(test_loader), 0):   
        rgb_gt = data_test[0].numpy().squeeze().transpose((1,2,0))
        rgb_noisy, mask = expand2square(data_test[1].cpu(), factor=128, ps=test_ps) 
        filenames = data_test[2]

        rgb_restored = model_testing(rgb_noisy.cuda())
        rgb_restored = torch.masked_select(rgb_restored,mask.bool().cuda()).reshape(1,3,rgb_gt.shape[0],rgb_gt.shape[1])
        rgb_restored = torch.clamp(rgb_restored,0,1).cpu().numpy().squeeze().transpose((1,2,0))

        psnr = psnr_loss(rgb_restored, rgb_gt)
        #ssim = ssim_loss(rgb_restored, rgb_gt, multichannel=True)
        ssim = ssim_loss(rgb_restored, rgb_gt, channel_axis=2)
        psnr_val_rgb.append(psnr)
        ssim_val_rgb.append(ssim)
        utils.save_img(os.path.join(result_dir,filenames[0]+'.PNG'), img_as_ubyte(rgb_restored))
        with open(os.path.join(result_dir,'psnr_ssim.txt'),'a') as f:
            f.write(filenames[0]+'.PNG ---->'+"PSNR: %.4f, SSIM: %.4f] "% (psnr, ssim)+'\n')
psnr_val_rgb = sum(psnr_val_rgb)/len(test_dataset)
ssim_val_rgb = sum(ssim_val_rgb)/len(test_dataset)
print(f"PSNR : {(psnr_val_rgb):.4f}dB \nSSIM : {(ssim_val_rgb):.4f}% ")
with open(os.path.join(result_dir,'psnr_ssim.txt'),'a') as f:
    f.write("Arch: Uformer_B, PSNR: %.4f, SSIM: %.4f] "% (psnr_val_rgb, ssim_val_rgb)+'\n')

100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:18<00:00,  2.17it/s]

PSNR:47.0271dB 
SSIM:0.9960% 


## Result Summary

The Uformer model fro deblur is trained and tested succesffully and we have the following set of value;  
PSNR between input & ground truth  and between model pecdiction with input & ground befotr training, PSNR on train_data and on val_data after training, PSNR between trained model & ground truth brfore testing and PSNR between trained model & ground truth and SSIM between the trained model and ground truth after testing; with which we can evaluate the performance of the model 